# Convert `suite2p` binary files to tiff stacks

This is a helper notebook to convert the `suite2p` binary files to tiff stacks (in the same folder).

| Binary input file | Tiff output file | 
| --: | :-- | 
| `data.bin` | `chan1.tiff` | 
| `data_chan2.bin` | `chan2.tiff` |
| `data_raw.bin` | `chan1_raw.tiff` | 
| `data_chan2_raw.bin` | `chan2_raw.tiff` |

This can be run on any environment as long as it has the necessary packages. If not do:

```bash
pip install numpy tifffile tqdm
```

This is preferably running in parallel for faster conversion of multiple files.


In [ ]:
import os
import glob

import numpy as np
from tifffile import TiffWriter
from tqdm.notebook import tqdm
from tqdm.contrib.concurrent import process_map

In [ ]:
# detect ops files with a certain path pattern
# alternatively, define `ops_files` as a list pointing to the appropriate `ops.npy` files
main_dir = '/oscar/data/afleisc2/collab/multiday-reg/data/SD_*'

ops_files = sorted(glob.glob(os.path.join(main_dir, '**/ops.npy'), recursive=True))
ops_files

In [ ]:
len(ops_files)

In [ ]:
def bin2tiff(bin_file, tiff_file, ops_file, tqdm_kwargs=dict()):
    """Convert a binary mmap file to tiff file from suite2p"""
    # metadata
    ops = np.load(ops_file, allow_pickle=True).item()
    n_frames, Ly, Lx = ops['nframes'], ops['Ly'], ops['Lx']

    # read in binary file
    memmap_obj = np.memmap(
        bin_file,
        mode='r',
        dtype='int16',
        shape=(n_frames, Ly, Lx)
    )
    
    # write to tiff file
    with TiffWriter(tiff_file, bigtiff=True) as f:
        for i in tqdm(range(n_frames), **tqdm_kwargs):
            curr_frame = np.floor(memmap_obj[i]).astype(np.int16)
            f.write(curr_frame, contiguous=True)
    
    # close binary file
    memmap_obj._mmap.close()
    

In [ ]:
def bin2tiff_one_ops(ops_file):
    """Conversion function for parallel"""
    
    ops_dir = os.path.dirname(ops_file)
    fluo_files = {
        'data.bin': 'chan1.tiff', 
        'data_chan2.bin': 'chan2.tiff',
        'data_raw.bin': 'chan1_raw.tiff', 
        'data_chan2_raw.bin': 'chan2_raw.tiff',
    }
    
    for bin_file, tiff_file in fluo_files.items():
        bin_file = os.path.join(ops_dir, bin_file)
        tiff_file = os.path.join(ops_dir, tiff_file)
        if (
            (not os.path.exists(bin_file))
            or os.path.exists(tiff_file)
        ):
            continue
        
        bin2tiff(
            bin_file,
            tiff_file,
            ops_file,
            tqdm_kwargs=dict(
                disable = True
            )
        )

process_map(
    bin2tiff_one_ops,
    ops_files,
    max_workers=16
)
    